In [1]:
from numpy.linalg import eig
import numpy as np
import cv2
import os
import sys
import pickle

In [2]:
configuration_file = 'config/part_1.cfg'

**Main: process_video.py**

In [3]:
from src.extract_features import *
from src.parsing import *
from src.display_video import *
from process_video import *
from compute_transform import*

config_data = parse_configuration_file(configuration_file) #Parse the configuration file
match_img1 , match_map = parse_points(config_data) #Parse the points from the configuration file
video_path, type_homography, file_name_keypoints, file_name_tranformations, \
    width, height = final_parsing_compute_transform(config_data)

H_frame1_to_map =compute_homography(match_img1, match_map)    
sift_points, nr_points = extract_keypoints(file_name_keypoints)
print("width, height", width, height)
match = matching_features_SCIKITLEARN(sift_points)
H_sequential = create_sequential_homographies(match, sift_points)



image matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')]
map matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')] 

width, height 804 454


In [4]:
#H_all = homography_to_map(H_sequential,height, width, sift_points)
H_tomap,Htoframe1 = homography_to_map(H_sequential, H_frame1_to_map,height, width, sift_points)


Output of process_video.py:

In [5]:
from src.outputs import *
create_output_keypoints(sift_points, file_name_keypoints, nr_points) #teste


In [6]:
from scipy.io import loadmat

def extract_keypoints(file_path):
    data = loadmat(file_path)
    keypoint_data = data['Keypoints']
    kp_list = []
    nr_points = None

    for item in keypoint_data:
        points = item.reshape((item.shape[1],item.shape[0])) # Transpose to revert the reshape operation
        kp_list.append(points)
        # Store the number of points (assuming all arrays have the same size)
        nr_points = points.shape[1] if nr_points is None else nr_points
    return kp_list, nr_points

sift_points_extracted, nr_points_extracted = extract_keypoints(file_name_keypoints)


**Main: compute_transform.py**

In [7]:
from src.matching_features import *
from src.homography import *
from src.ransac import *
from src.view_homography import *
from compute_transform import *

def create_sequential_homographies(matches, sift_points):
    """ This function creates the homographies from frame n+1 to frame n.
        We want the homography from frame n+1 to frame n because we will 
        want the homography from any frame back to the frame 1 and then to the map."""
    H_sequential=np.empty([11,0])
    for i in range(len(matches)):
        kp_dst = sift_points[i][:2,:] #keyupoints for frame n (destination)
        kp_src= sift_points[i+1][:2,:] #keypoints for frame n+1 (origin)
        src_pts = [] #[[x1,y1, d1, ..., dn], [x1,y1, d1, ..., dn], ...]
        dst_pts = []
        for k in matches[i][0,:]:
            src_pts.append(  (float(kp_src[0,int(k)])   , float(kp_src[1,int(k)]))   )
        for k in matches[i][1,:]:
            dst_pts.append(  (float(kp_dst[0,int(k)])   , float(kp_dst[1,int(k)]))   )

        H_parameters, inliers = RANSAC(src_pts, dst_pts, 72, 0.8) #71.36 iterations to 0.99 suceess-> for 50% inliers
        indexes_frames = np.array([[i+1], [i+2]])
        H = np.vstack((indexes_frames, H_parameters.reshape(9,1) ))
        H_sequential = np.hstack([H_sequential, H])
        
    return H_sequential

video_path, type_homography, file_name_keypoints, file_name_tranformation, width, height = final_parsing_compute_transform(config_data)
H_frame1_to_map =compute_homography(match_img1, match_map)    
match = matching_features_SCIKITLEARN(sift_points)
H_sequential = create_sequential_homographies(match, sift_points)
if type_homography =='map':
    H_output = homography_to_map(H_sequential, H_frame1_to_map)
elif type_homography =='all':
    H_output = all_homographies(H_sequential)
    
H_output = recalculate_homographies_if_intersection(H_output, int(height), int(width), sift_points, match)

index_frame_i, index_frame_j 2 1
index_frame_i, index_frame_j 3 1
index_frame_i, index_frame_j 4 1
index_frame_i, index_frame_j 5 1
index_frame_i, index_frame_j 6 1
index_frame_i, index_frame_j 7 1
index_frame_i, index_frame_j 8 1
index_frame_i, index_frame_j 9 1
index_frame_i, index_frame_j 10 1
index_frame_i, index_frame_j 11 1
index_frame_i, index_frame_j 12 1
index_frame_i, index_frame_j 13 1
index_frame_i, index_frame_j 14 1
index_frame_i, index_frame_j 15 1
index_frame_i, index_frame_j 16 1
index_frame_i, index_frame_j 17 1
index_frame_i, index_frame_j 18 1
index_frame_i, index_frame_j 19 1
index_frame_i, index_frame_j 3 2
index_frame_i, index_frame_j 4 2
index_frame_i, index_frame_j 5 2
index_frame_i, index_frame_j 6 2
index_frame_i, index_frame_j 7 2
index_frame_i, index_frame_j 8 2
index_frame_i, index_frame_j 9 2
index_frame_i, index_frame_j 10 2
index_frame_i, index_frame_j 11 2
index_frame_i, index_frame_j 12 2
index_frame_i, index_frame_j 13 2
index_frame_i, index_frame_j 

Output of compute_transform.py:

In [8]:
from src.outputs import create_output
i = create_output(H_output, file_name_tranformations)        



**Display results:**

In [5]:
def show_pixel_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at (x={x}, y={y})")

def display_frame(name,frame):
    cv2.namedWindow(name)
    cv2.setMouseCallback(name, show_pixel_coordinates)
    cv2.imshow(name, frame)

def display(frame1, frame2,homography_de2_para1 , idx1, idx2 ):
    H= homography_de2_para1.reshape((3,3))
    
    height, width = frame2.shape[:2]
    # Apply homography to frame 2
    warped_frame2 = cv2.warpPerspective(frame2, H, (width, height))

    while True:
        display_frame(f'Frame {idx1}', frame1)

        display_frame(f'Frame{ idx2} transformed with homography to coordinates of frame {idx1}', warped_frame2)

        display_frame(f"Frame {idx2}", frame2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(1) & 0xFF == ord('e'):
            return True
    cv2.destroyAllWindows()
    return False

def extract__frames(video_path):
    """Extracts the features from the video and stores them in a list"""
    print(video_path)
    capture = cv2.VideoCapture(os.path.abspath(video_path))
    k = 0
    frames=[]
    count_frames(video_path)
    while k <= 1900:
        capture.set(cv2.CAP_PROP_POS_FRAMES, k)
        success, frame = capture.read() #read the video
        if success:
            frames.append(frame)
        k += 100
    return frames

In [6]:

frames= extract__frames('video/trymefirst_lisbon.mp4')

video/trymefirst_lisbon.mp4
Total frames of the video:  1901


In [7]:
#to test homography to map
img1=frames[0]
idx1 = 0
print('Press q to change image\nPress e to exit')

for i in range(1,len(frames)-1):
    idx2 = i 
    img2= frames[i]
    exit_bool = display(img1, img2, Htoframe1[2:,i], idx1, idx2 )
    if exit_bool:
        cv2.destroyAllWindows()
        break

Press q to change image
Press e to exit
Clicked at (x=386, y=151)
Clicked at (x=238, y=109)
Clicked at (x=522, y=152)
Clicked at (x=611, y=267)
Clicked at (x=479, y=228)
Clicked at (x=657, y=402)
Clicked at (x=660, y=194)
Clicked at (x=490, y=79)
Clicked at (x=475, y=143)
Clicked at (x=446, y=134)


In [10]:
#to test homography to each other
print('Press q to change image\nPress e to exit')
#H_output = all_homographies(H_sequential)

for i in range(H_output.shape[1]):
    idx1= int(H_output[0,i])
    idx2= int(H_output[1,i])

    img1 = frames[ idx1-1 ] 
    img2= frames[  idx2-1]
    exit_bool = display(img1, img2, H_output[2:,i], idx1, idx2 )
    if exit_bool:
        cv2.destroyAllWindows()
        break

Press q to change image
Press e to exit
Clicked at (x=327, y=103)
Clicked at (x=526, y=235)
Clicked at (x=425, y=108)
Clicked at (x=499, y=227)
Clicked at (x=342, y=80)
Clicked at (x=222, y=161)
Clicked at (x=301, y=122)
Clicked at (x=149, y=104)
Clicked at (x=114, y=109)
Clicked at (x=106, y=75)
